In [ ]:
# -*- coding: utf-8 -*-
# This code is written in Python 3.7. To install the required packages, run the following commands:
# pip install pandas numpy matplotlib seaborn scikit-learn sympy
# This code is applicable to the Simargl 2022 dataset.
# implemented Simple Ensemble Learning (Averaging Classifier) using Decision Tree, K-Nearest Neighbors, and Random Forest

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load the cleaned Simargl 2022 dataset
df = pd.read_csv(
    '/home/ibibers@ads.iu.edu/IDS_Datasets/Combined_datasets/Simargl_cleaned_dataset.csv')

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Extract subsample of data
print(" Extract subsample of data: ")
print(df['ALERT'].value_counts())

In [ ]:
dropped_df = ['ALERT']
X = df.drop(dropped_df, axis=1)
y = df['ALERT']

# Label encoding for the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

print(df['ALERT'].value_counts())

In [ ]:
# Initial train test split set and split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Convert data to TensorFlow tensors
X_train_tf = tf.convert_to_tensor(X_train.values, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(pd.factorize(y_train)[0], dtype=tf.int64)
X_test_tf = tf.convert_to_tensor(X_test.values, dtype=tf.float32)

# Initialize MirroredStrategy for GPU acceleration
strategy = tf.distribute.MirroredStrategy()

 # Train the models
with strategy.scope():
    # Initialize the models inside the strategy scope
    model1 = DecisionTreeClassifier()
    model2 = KNeighborsClassifier()
    model3 = RandomForestClassifier()

    model1.fit(X_train, y_train)
    model2.fit(X_train, y_train)
    model3.fit(X_train, y_train)

    # Predict the models
    pred1 = model1.predict_proba(X_test)
    pred2 = model2.predict_proba(X_test)
    pred3 = model3.predict_proba(X_test)

# Convert probabilities to TensorFlow tensors
pred1_tf = tf.convert_to_tensor(pred1, dtype=tf.float32)
pred2_tf = tf.convert_to_tensor(pred2, dtype=tf.float32)
pred3_tf = tf.convert_to_tensor(pred3, dtype=tf.float32)

In [ ]:
# Final prediction using Simple Average (Soft Voting)
finalpred = (pred1_tf + pred2_tf + pred3_tf) / 3

# If finalpred is a 1D array, convert it to a 2D array
if finalpred.ndim == 1:
    finalpred = finalpred[:, np.newaxis]

# Convert probabilities to class predictions
final_class_pred_enc = np.argmax(finalpred, axis=1)

# Evaluate the model
accuracy = accuracy_score(y_test, final_class_pred_enc)
print(f"Accuracy: {accuracy}")


In [ ]:
# Use inverse_transform to get original labels
y_pred = label_encoder.inverse_transform(final_class_pred_enc)
y_test_labels = label_encoder.inverse_transform(y_test)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))

In [ ]:

# Evaluate the model
accuracy = accuracy_score(y_test, final_class_pred_enc)
print(f"Accuracy: {accuracy}")

In [ ]:

# Confusion matrix with string labels
conf_mat = confusion_matrix(y_test_labels, y_pred)

# Get unique class labels from y_test and y_pred_encoded
unique_labels = np.unique(np.concatenate(
    (y_test_labels, y_pred)))

# Plotting the Confusion Matrix with class labels
plt.figure(figsize=(6, 6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=unique_labels, yticklabels=unique_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()